In [54]:
import yfinance as yf
import utils as ut
from pandas_datareader import data as pdr
import pandas as pd
from openpyxl import load_workbook


In [55]:
ut.property_list(yf.Ticker)

['actions', 'balance_sheet', 'balancesheet', 'calendar', 'cashflow', 'dividends', 'earnings', 'financials', 'info', 'institutional_holders', 'isin', 'major_holders', 'options', 'quarterly_balance_sheet', 'quarterly_balancesheet', 'quarterly_cashflow', 'quarterly_earnings', 'quarterly_financials', 'recommendations', 'splits', 'sustainability']


In [56]:
metriclist=['MarketCap','PERatio_TTM','EPS_TTM','CurrentRatio', 'DebtRatio', 'ROS', 'ROE']
## Initialization
# watchlist_dic = {'INFORMATION TECHNOLOGY':['AAPL','MSFT','GOOG','FB'],
#                  'CONSUMER DISCRETIONARY':['AMZN','HD','DIS'],
#                  'FINANCIALS':['BRK-B','JPM','BAC'], 
#                  'REAL ESTATE':['AMT','PLD','SPG'],
#                  'CONSUMER STAPLES':['WMT','PG','KO'],
#                  'INDUSTRIALS':['UPS','LMT','BA'],
#                  'ENERGY':['XOM','CVX','OXY'],
#                  'HEALTH CARE':['JNJ','PFE','GILD']}

## Please add the target stock code directly
watchlist_dic = {'INFORMATION TECHNOLOGY':[],
                 'CONSUMER DISCRETIONARY':[],
                 'FINANCIALS':[], 
                 'REAL ESTATE':[],
                 'CONSUMER STAPLES':[],
                 'INDUSTRIALS':[],
                 'ENERGY':[],
                 'HEALTH CARE':[]}
table_dic={}

In [57]:
for key in watchlist_dic:
#     print(key)
    watchlist=watchlist_dic[key]
    tickers = yf.Tickers(watchlist)
    moneyList = pd.DataFrame(columns=metriclist)
    try:
        for index,stock_ID in enumerate(tickers.symbols): 
            ticker=tickers.tickers[index]
            institutionalHolders=ticker.institutional_holders.iloc[:,1]
            MarketCap=institutionalHolders[0]
            PERatio_TTM=institutionalHolders[2]
            EPS_TTM=institutionalHolders[3]
            incomeStatement=ticker.financials.iloc[:,0]
            balanceSheet=ticker.balancesheet.iloc[:,0]
            cashFlow=ticker.cashflow.iloc[:,0]
            CurrentRatio=balanceSheet.loc['Total Current Assets']/balanceSheet.loc['Total Current Liabilities']
            DebtRatio=balanceSheet.loc['Total Liab']/balanceSheet.loc['Total Assets']
            ROS=incomeStatement.loc['Net Income']/incomeStatement.loc['Total Revenue']
            ROE=incomeStatement.loc['Net Income']/balanceSheet.loc['Total Stockholder Equity']
            moneyList.loc[stock_ID] = {'MarketCap':MarketCap,'PERatio_TTM':PERatio_TTM,'EPS_TTM':EPS_TTM,'CurrentRatio':CurrentRatio, 'DebtRatio':DebtRatio, 'ROS': ROS, 'ROE': ROE}
        table_dic[key] = moneyList
    except (KeyError,IndexError) as err:
            print (err,key,stock_ID)         

In [58]:
# book = load_workbook('1.xlsx')
# with pd.ExcelWriter('1.xlsx',mode='a',engine='openpyxl') as writer: 
#     for key in table_dic:
#         ut.append_df_to_Excel('1.xlsx', table_dic[key], sheet_name=key)
#         print(key)
#         writer.book = book
#         writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
#         table_dic[key].to_excel(writer, sheet_name=key)
#     writer.save()
# writer.close()

In [59]:
for key in table_dic:
        ut.append_df_to_Excel('watchlist.xlsx', table_dic[key], sheet_name=key,float_format="%.3f")